In [ ]:
import tables
import h5py
import numpy as np
import sys
from pyhdf.SD import SD, SDC
import pandas as pd
import time
import calendar
import matplotlib.pyplot as plt
import datetime
import os
import openturns as ot

## Initial Exploration

In [ ]:
#change this obv
test = pd.read_hdf("modis_aod/modis_aod_regridded/regridded_2018.h5")

In [ ]:
test

,0,1,2,3,4,5,6,7,8,9,...,14390,14391,14392,14393,14394,14395,14396,14397,14398,14399
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,173.0,145.0,0.0,80.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,26.0,41.5,45.000000,43.0,0.0,60.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
361,0.0,59.0,57.666667,73.0,70.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
362,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,96.0,78.0,72.0,60.0,53.5,71.0,78.25,0.0,0.0,105.0
363,271.5,253.0,230.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,66.0,0.0,75.0,0.0,48.5,0.00,0.0,0.0,0.0


In [ ]:
#one day
krig_data = test.iloc[215]

In [ ]:
krig_data

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
         ...  
14395    733.0
14396    495.0
14397      0.0
14398      0.0
14399      0.0
Name: 215, Length: 14400, dtype: float64

In [ ]:
#
krig_data_train = np.where(krig_data != 0) #np.where(hum != 0)
krig_data_test = np.where(krig_data == 0)

In [ ]:
krig_data_train = krig_data[krig_data_train[0]] #gets values
krig_data_test = krig_data[krig_data_test[0]]

In [ ]:
krig_data_train.index[5]

57

119.0

495.0

## Kriging Training for 8/4/2018 (216th day of 2018)

In [ ]:
lons = [krig_data_train.index[i] % 120 for i in range(len(krig_data_train))]

In [ ]:
lats = [(krig_data_train.index[i] - (krig_data_train.index[i] % 120)) / 120 for i in range(len(krig_data_train))]

In [ ]:
c = [np.array(i) for i in zip(lats, lons)]
d = [[krig_data_train.iloc[i]] for i in range(len(krig_data_train))]

In [ ]:
coordinates_train = ot.Sample(c)
aod_train = ot.Sample(d)

In [ ]:
# Actual Training Step

inputDimension = 2
basis = ot.ConstantBasisFactory(inputDimension).build()
covarianceModel = ot.SquaredExponential([1.]*inputDimension, [1.0])
algo = ot.KrigingAlgorithm(coordinates_train, aod_train, covarianceModel, basis)
algo.run()
result = algo.getResult()
krigingMetamodel = result.getMetaModel()

## Predicting

In [ ]:
lons_test = [krig_data_test.index[i] % 120 for i in range(len(krig_data_test))]
lats_test = [(krig_data_test.index[i] - (krig_data_test.index[i] % 120)) / 120 for i in range(len(krig_data_test))]
c = [np.array(i) for i in zip(lats_test, lons_test)]

In [ ]:
kriged_data_pred = krigingMetamodel(c)

In [ ]:
kriged_data_pred = pd.Series(np.array(kriged_data_pred).reshape(-1))

## Final Array with Existing and Predicted AOD

In [ ]:
kriged_map_arr = np.zeros((120, 120))

for i in range(len(krig_data_test.index)):
    kriged_map_arr[(int)((krig_data_test.index[i] - (krig_data_test.index[i] % 120)) / 120), (int)(krig_data_test.index[i] % 120)] = kriged_data_pred[i]
    
for i in range(len(krig_data_train.index)):
    kriged_map_arr[(int)((krig_data_train.index[i] - (krig_data_train.index[i] % 120)) / 120), (int)(krig_data_train.index[i] % 120)] = krig_data_train[krig_data_train.index[i]]

In [ ]:
sum(sum(kriged_map_arr == 0))

0

In [ ]:
kriged_map_pd = pd.DataFrame(kriged_map_arr)

In [ ]:
kriged_map_pd.to_hdf("modis_aod/modis_aod_regridded/krig_map.h5", key="df")

## Mapping (do in separate notebook for AOD, kernel crashes when done in this notebook)

In [ ]:
fig, ax = plt.subplots(figsize = (5,5))
test_map = ax.imshow(kriged_map_arr, cmap="hot", interpolation="none")
cbar = fig.colorbar(test_map, ax=ax)

NameError: name 'plt' is not defined